In [61]:
import pandas as pd

# Load and prepare the data
data = pd.read_csv('data/us30.csv')
data['timestamp'] = pd.to_datetime(data['Local time'])
data = data[['timestamp', 'Open', 'High', 'Low', 'Close']]
data.rename(columns={'Close': 'price'}, inplace=True)

tick_size = 0.01

# Define the strategy function
def trading_strategy(data):
    trades = []
    data['date'] = data['timestamp'].dt.date
    data['time'] = data['timestamp'].dt.time
    
    # Iterate over each day in the dataset
    for day in data['date'].unique():
        day_data = data[data['date'] == day]
        
        # Get the closing price from the previous day at 16:00
        previous_day = pd.to_datetime(day) - pd.Timedelta(days=1)
        previous_day_data = data[data['date'] == previous_day.date()]
        if previous_day_data.empty:
            continue
        close_yesterday_data = previous_day_data[previous_day_data['time'] == pd.to_datetime('16:00').time()]
        if close_yesterday_data.empty:
            continue
        close_yesterday = close_yesterday_data.iloc[0]['price']
        
        # Find the entry time, which is the closest time after 7:05 UTC
        entry_time = day_data[day_data['time'] >= pd.to_datetime('07:05').time()].iloc[0]
        entry_price = entry_time['price']
        entry_timestamp = entry_time['timestamp']
        
        # Determine direction (long or short)
        if entry_price > close_yesterday:
            direction = 1  # Long
        else:
            direction = -1  # Short
            
        # Set initial SL and TP
        sl = entry_price - direction * (350 * tick_size)
        tp = entry_price + direction * (350 * tick_size)
        
        # Define function to handle reentry logic after SL hit
        def reenter_trade(entry_price, direction, r):
            new_sl = entry_price - direction * (350 * tick_size)  # SL is always 350 ticks away
            new_tp = entry_price + direction * (350 * r * tick_size)  # TP is r * 350 ticks away
            return new_sl, new_tp

        trade_pnl = 0
        stop_trading = False  # New variable to stop further trading once a successful trade is made
        reentry_count = 0  # Counter for reentries, max of 2 reentries (1R -> 2R -> 3R)

        # Check the price movements after the entry
        for i, row in day_data[day_data['time'] >= pd.to_datetime('07:05').time()].iterrows():
            if stop_trading:
                break  # Exit loop if a profitable trade was already made

            current_price = row['price']
            current_time = row['timestamp']
            high_price = row['High']
            low_price = row['Low']

            # Check for Stop Loss (based on Low for long, High for short)
            if direction == 1 and low_price <= sl:  # Long position SL hit
                trade_pnl = (sl - entry_price) / entry_price
                trades.append({'entry_time': entry_timestamp, 'exit_time': current_time, 'pnl_pct': trade_pnl, 'R': reentry_count + 1})

                reentry_count += 1  # Increase the reentry count
                if reentry_count == 3:
                    # Stop after 3R SL hit
                    stop_trading = True
                    break

                # Reenter trade at SL price with adjusted 2R or 3R setup
                entry_price = sl  # Reenter at the SL price
                entry_timestamp = current_time
                sl, tp = reenter_trade(entry_price, direction, reentry_count + 1)  # Adjust for 2R or 3R setup
                continue

            elif direction == -1 and high_price >= sl:  # Short position SL hit
                trade_pnl = (entry_price - sl) / entry_price
                trades.append({'entry_time': entry_timestamp, 'exit_time': current_time, 'pnl_pct': trade_pnl, 'R': reentry_count + 1})

                reentry_count += 1  # Increase the reentry count
                if reentry_count == 3:
                    # Stop after 3R SL hit
                    stop_trading = True
                    break

                # Reenter trade at SL price with adjusted 2R or 3R setup
                entry_price = sl  # Reenter at the SL price
                entry_timestamp = current_time
                sl, tp = reenter_trade(entry_price, direction, reentry_count + 1)  # Adjust for 2R or 3R setup
                continue
            
            # Check for Take Profit (based on High for long, Low for short)
            if direction == 1 and high_price >= tp:  # Long position TP hit
                trade_pnl = (tp - entry_price) / entry_price
                trades.append({'entry_time': entry_timestamp, 'exit_time': current_time, 'pnl_pct': trade_pnl, 'R': reentry_count + 1})
                stop_trading = True  # Stop trading for the day after a profitable trade
                break
            elif direction == -1 and low_price <= tp:  # Short position TP hit
                trade_pnl = (entry_price - tp) / entry_price
                trades.append({'entry_time': entry_timestamp, 'exit_time': current_time, 'pnl_pct': trade_pnl, 'R': reentry_count + 1})
                stop_trading = True  # Stop trading for the day after a profitable trade
                break
            
            # Check if 1 hour has passed without hitting SL or TP
            if (current_time - entry_timestamp).total_seconds() >= 3600:
                break
    
    trades_df = pd.DataFrame(trades)
    return trades_df

# Apply the strategy to the data
trades_df = trading_strategy(data)




In [62]:
trades_df.head(50)


,entry_time,exit_time,pnl_pct,R
0,2024-08-14 07:05:00-02:00,2024-08-14 07:18:00-02:00,0.000088,1
1,2024-08-15 07:05:00-02:00,2024-08-15 07:06:00-02:00,0.000087,1
2,2024-08-16 07:05:00-02:00,2024-08-16 07:07:00-02:00,0.000086,1
3,2024-08-20 07:05:00-02:00,2024-08-20 07:07:00-02:00,-0.000086,1
4,2024-08-20 07:07:00-02:00,2024-08-20 07:18:00-02:00,0.000171,2
5,2024-08-21 07:05:00-02:00,2024-08-21 07:06:00-02:00,-0.000086,1
6,2024-08-21 07:06:00-02:00,2024-08-21 07:07:00-02:00,-0.000086,2
7,2024-08-21 07:07:00-02:00,2024-08-21 07:15:00-02:00,0.000257,3
8,2024-08-22 07:05:00-02:00,2024-08-22 07:08:00-02:00,0.000086,1
9,2024-08-23 07:05:00-02:00,2024-08-23 07:20:00-02:00,-0.000086,1


In [63]:
trades_df.pnl_pct.describe()

count    22.000000
mean     -0.000004
std       0.000119
min      -0.000086
25%      -0.000085
50%      -0.000085
75%       0.000086
max       0.000257
Name: pnl_pct, dtype: float64